In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import KFold, StratifiedKFold
import sklearn.metrics
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [ ]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model,load_model

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

In [ ]:
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO
import collections
import random
import os
import time
import json
from PIL import Image
import math

In [ ]:
# Mouting drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Link to the provided data
trainPath='/content/drive/MyDrive/steelDefects/train'
testPath='/content/drive/MyDrive/steelDefects/test'

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(9, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
testSize=40
batchSize=15
epochs=200
targetSize=(300,300)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3,
                                   rotation_range=50,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   brightness_range = [0.4, 1.2],
                                   fill_mode='nearest',
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255, rotation_range=50,)

In [ ]:
augmentedTrain = [train_generator[0][0][j] for j in range(9) for i in range(16)]
plotImages(augmentedTrain)

In [ ]:
augmentedTest = [validation_generator[0][0][j] for j in range(9) for i in range(16)]
plotImages(augmentedTest)

In [ ]:
train_generator = train_datagen.flow_from_directory(
                  trainPath,
                  target_size=targetSize,
                  batch_size=batchSize,
                  class_mode='categorical',
                  subset='training')

validation_generator = train_datagen.flow_from_directory(
                       trainPath,
                       target_size=targetSize,
                       batch_size=batchSize,
                       class_mode='categorical',
                       subset='validation')

In [ ]:
def CVRun(gaussianProcess, NoOfSplits, X, y):
  kf = KFold(n_splits=NoOfSplits)

  # For tracking results of each split
  errors = []
  kernels = []

  # Iterating over all splits
  for idx, (train, val) in enumerate(kf.split(X)):
      # Seperate the parts of the data
      CV_XTrain = X.values[train]
      CV_XVal = X.values[val]
      CV_YTrain = y.values[train]
      CV_YVal = y.values[val]

      # Normalise the dataset
      CV_XTrain = scaler.fit_transform(CV_XTrain)
      CV_XVal = scaler.transform(CV_XVal)

      # Model fit and prediction
      model = gaussianProcess.fit(CV_XTrain, CV_YTrain)
      Predicted_YTrain = model.predict(CV_XTrain)
      Predicted_YVal = model.predict(CV_XVal)


      # Computing errors
      rmse_val = np.sqrt(mean_squared_error(CV_YVal, Predicted_YVal))

      errors.append(rmse_val)
      kernels.append(model.kernel_)

  # Find and return the mean optimal parameter and error values over all splits
  sigma_cs = []
  sigma_ls = []

  for k in kernels:
      sigma_cs.append(k.k1.constant_value)
      sigma_ls.append(k.k2.length_scale)
  k_best = np.median(sigma_cs) * RBF(length_scale=np.median(sigma_ls), length_scale_bounds=(1e-4, 1e2))

  print(errors)
  print(pd.DataFrame(errors).mean()[0])

  return pd.DataFrame(errors).mean()[0], k_best